Scraping Player Information of Chelsea Football Club

In [1]:
import pandas as pd
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import matplotlib.pyplot as plt
import os
import logging

In [2]:
chrome_options = Options()
chrome_options.add_argument("--headless") # Ensure GUI is off
chrome_options.add_argument("--no-sandbox")

# Silent download of drivers
logging.getLogger('WDM').setLevel(logging.NOTSET)
os.environ['WDM_LOG'] = 'False'

# Create service
webdriver_service = Service(ChromeDriverManager().install())

# Create driver
driver = webdriver.Chrome(service = webdriver_service, options = chrome_options)

In [3]:
url = 'https://www.chelseafc.com/en/teams/men?tab=mensteam'
driver = webdriver.Chrome()
driver.get(url)

In [4]:
acceptXpath  = '//*[@id="onetrust-accept-btn-handler"]'
time.sleep(3)
driver.find_element(by=By.XPATH, value=acceptXpath).click()


In [5]:
player_elements = driver.find_elements(by=By.CLASS_NAME, value='listing__item')
players = []
detailLinks =[]
for p in player_elements:
        try :
                firstName = p.find_element(by=By.CLASS_NAME, value='player-card__first-name').text
                lastName = p.find_element(by=By.CLASS_NAME, value='player-card__last-name').text
                position = p.find_element(by=By.CLASS_NAME, value='player-card__position').text
                number = p.find_element(by=By.CLASS_NAME, value='player-card__number').text
                players.append({'firstName': firstName, 'lastName': lastName, 'position': position, 'number': number})

                l = p.find_element(By.CSS_SELECTOR, "a.link.player-card__link.floating-link").get_attribute('href')
                detailLinks.append(l)                
        except: 
                name = p.find_element(by=By.CLASS_NAME, value='player-card__known-as').text
                firstName = name.split(' ')[0]
                lastName = name.split(' ')[1]
                position = p.find_element(by=By.CLASS_NAME, value='player-card__position').text
                number = p.find_element(by=By.CLASS_NAME, value='player-card__number').text
driver.quit()
pd.DataFrame(players)

,firstName,lastName,position,number
0,Robert,SANCHEZ,Goalkeeper,1
1,Filip,JORGENSEN,Goalkeeper,12
2,Marcus,BETTINELLI,Goalkeeper,13
3,Lucas,BERGSTROM,Goalkeeper,47
4,Axel,DISASI,Centre-back,2
5,Marc,CUCURELLA,Defender/wing-back,3
6,Tosin,ADARABIOYO,Centre-back,4
7,Benoit,BADIASHILE,Centre-back,5
8,Levi,COLWILL,Centre-back,6
9,Ben,CHILWELL,Left-back/left wing-back,21


In [6]:
players_detail = []
for l in detailLinks:
    drv = webdriver.Chrome()
    drv.get(l)
    acceptXpath  = '//*[@id="onetrust-accept-btn-handler"]'
    time.sleep(2)
    try:
        drv.find_element(by=By.XPATH, value=acceptXpath).click()
    except: pass

    d = {}
    try:
        detail_labels = drv.find_elements(by=By.CLASS_NAME, value='profile-player-details__item-label')
        detail_values = drv.find_elements(by=By.CLASS_NAME, value='profile-player-details__item-value')
        
        for i in range(len(detail_labels)):
            d[detail_labels[i].text.replace(":",'')] = detail_values[i].text
    except: 
        d={'Err':'err'}
    drv.quit()
    players_detail.append(d)
pd.DataFrame(players_detail)

,NAME,NATIONAL TEAM,PLACE OF BIRTH,POSITION,DOB,HEIGHT,DEBUT,NATIONALITY
0,Robert Sanchez,Spain,"Cartagena, Spain",Goalkeeper,"18 November, 1997 (age 26)",1.97m (6 ft 6 in),"13 August, 2023",NaN
1,Filip Jorgensen,Denmark Under-21s,"Lomma, Sweden",Goalkeeper,"16 April, 2002 (age 22)",1.92m (6ft 4in),"22 August, 2024",NaN
2,Marcus Bettinelli,NaN,"London, England",Goalkeeper,"24 May, 1992 (age 32)",1.93m (6 ft 4 in),"8 January, 2022",England
3,Lucas Bergstrom,Finland,"Pargas, Finland",Goalkeeper,"5 September, 2002 (age 22)",2.05m (6ft 9in),NaN,NaN
4,Axel Disasi,France,"Gonesse, France",Centre-back,"11 March, 1998 (age 26)",1.90m (6ft 3in),"13 August, 2023",NaN
5,Marc Cucurella,Spain,"Alella, Spain",Defender/wing-back,"22 July, 1998 (age 26)",1.73m (5 ft 8 in),"6 August, 2022",NaN
6,Tosin Adarabioyo,NaN,"London, England",Centre-back,"24 September, 1997 (age 27)",1.96m (6ft 5in),NaN,England
7,Benoit Badiashile,France,"Limoges, France",Centre-back,"26 March, 2001 (age 23)",1.94m (6 ft 4 in),"15 January, 2023",NaN
8,Levi Colwill,England,"Southampton, England",Centre-back,"26 February, 2003 (age 21)",1.87m (6 ft 2 in),"13 August, 2023",NaN
9,Ben Chilwell,England,"Milton Keynes, England",Left-back/left wing-back,"21 December, 1996 (age 27)",1.81m (5 ft 11 in),"23 September, 2020",NaN
